### 라이브러리 로드

In [37]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import csv
from pprint import pprint # Json 출력 인덴트 주어 보기 편하게 출력.
import pandas as pd # 데이터프레임 가공을 위한 라이브러리.
import psycopg2 # 데이터베이스 연동을 위한 라이브러리.

### DB 불러오기

In [3]:
db = psycopg2.connect(host='', dbname='',user='',password='',port=) # 파이썬 db 연동.

### Table 불러오기

In [7]:
pheno = pd.read_sql("""select pheno_id, pheno_origin, pheno, category, object_, adverb, comment_, writer from munjin.pheno2 order by pheno_id;""", db)

* pheno2 테이블

### Dict records 형태 변환

In [9]:
pheno_dict_records = pheno.to_dict('records')

---------

### 엘라스틱 연동

In [10]:
es = Elasticsearch("")
es.info()

{'name': '',
 'cluster_name': '',
 'cluster_uuid': '',
 'version': {'number': '7.3.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '4749ba6',
  'build_date': '2019-08-19T20:19:25.651794Z',
  'build_snapshot': False,
  'lucene_version': '8.1.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### 인덱스 Settings & Mappings 설정

In [11]:
index_sm = {
"settings":{
    "analysis":{
        "analyzer":{
            "nori":{
                "type": "custom",
                "tokenizer":"nori_tokenizer",
                "decompound":"mixed"
                }
            }
        }
    },
"mappings":{
    "properties":{
        "pheno_id":{
            "type": "integer"
            },
        "pheno_origin":{
            "type": "text",
            "analyzer": "nori"
            },
        "pheno":{
            "type": "text",
            "analyzer": "nori"
            },
        "category":{
            "type": "keyword"
            },
        "object_":{
            "type": "keyword"
            },
        "adverb":{
            "type": "keyword"
            },
        "comment_":{
            "type": "text",
            "analyzer": "nori"
            },
        "writer":{
            "type": "keyword"
            }
        }
    }        
}


### INDEX 생성 함수

In [12]:
def create_index(es, index, body=None):
    if not es.indices.exists(index=index):
        if body is None:
            r = es.indices.create(index=index)
        else:
            r = es.indices.create(index=index, body=body)
            
        return r
    
    else:
        return "Index already exists"

### INDEX 생성

In [13]:
create_index(es, "pheno", index_sm)

<ipython-input-12-2aed582c6c1a>:6: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  r = es.indices.create(index=index, body=body)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'pheno'}

### Generator 함수(Helpers.bulk 함수 적용 위함)

In [33]:
def generator(data_dict_records):
    for i in range(0, len(data_dict_records)):
        yield {"_id":data_dict_records[i]['pheno_id'], "_source": data_dict_records[i]}

In [34]:
import_data = generator(pheno_dict_records)

### BULK DATA LOAD

In [35]:
helpers.bulk(es, import_data, index="pheno")

(1224, [])

### 인덱스 내 모든 doc 검색

In [36]:
es.search(index='pheno', body={"query":{"match_all" : {}}})

<ipython-input-36-48a991848474>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index='pheno', body={"query":{"match_all" : {}}})


{'took': 129,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1224, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'pheno',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0,
    '_source': {'pheno_id': 1,
     'pheno_origin': '어지러움',
     'pheno': '어지러움',
     'category': 'clinical finding',
     'object_': None,
     'adverb': None,
     'comment_': '',
     'writer': 'yoon'}},
   {'_index': 'pheno',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.0,
    '_source': {'pheno_id': 2,
     'pheno_origin': '기침',
     'pheno': '기침',
     'category': 'clinical finding',
     'object_': None,
     'adverb': None,
     'comment_': '',
     'writer': 'yoon'}},
   {'_index': 'pheno',
    '_type': '_doc',
    '_id': '3',
    '_score': 1.0,
    '_source': {'pheno_id': 3,
     'pheno_origin': '콧물',
     'pheno': '콧물',
     'category': 'clinical finding',
     'object_': None,
     'adverb': None,
     '